# Performing differential expression analysis between treatment group and DMSO from  subsampled data without replacement

In [1]:
library(edgeR)
library(parallel)

Loading required package: limma



Loading data: these objects are a list previously produced and containing as elements matrices with pseudobulk replicates obtained via bootstrapping

In [2]:
JHOS2_pseudo <- readRDS("JHOS2_pseudobulk_subsampled_REVIEW.RDS")
PDC2_pseudo <- readRDS("PDC2_pseudobulk_subsampled_REVIEW.RDS")
PDC3_pseudo <- readRDS("PDC3_pseudobulk_subsampled_REVIEW.RDS")

In [3]:
sapply(JHOS2_pseudo, function(x) ncol(x))
sapply(PDC2_pseudo, function(x) ncol(x))
sapply(PDC3_pseudo, function(x) ncol(x))

AR-42      AT 101      AT9283    AZD-8186     AZD8055  Belinostat 
         17           6          28          17          33          28 
    BI 2536  Birabresib  Birinapant  BMS-754807  Buparlisib Cobimetinib 
         28          30          22          28          28          32 
 Copanlisib  Dactolisib  Danusertib   Dasatinib  Dinaciclib        DMSO 
         28          33          25          33          27          83 
  ENMD-2076  Fedratinib Gedatolisib    I-BET151 Ipatasertib         JQ1 
         27          27          33          27          29          22 
  LY3009120   Milciclib   Niraparib  NVP-BGT226  NVP-LCL161      OTS167 
         17          29          30          36          26          29 
 Pacritinib   PD0325901 PF-00477736 PF-03758309  Pictilisib Pracinostat 
         25          30          26          24          29          28 
Quisinostat Ralimetinib   SCH772984   Selinexor     SNS-032 Talazoparib 
         30          33          29          32          32          26 
    TGX-221      UCN-01  Volasertib     ZSTK474 
         28          29          26          31

AR-42      AT 101      AT9283    AZD-8186     AZD8055  Belinostat 
         15          12          18          23          26          19 
    BI 2536  Birabresib  Birinapant  BMS-754807  Buparlisib Cobimetinib 
         24          27          27          15          23          34 
 Copanlisib  Dactolisib  Danusertib   Dasatinib  Dinaciclib        DMSO 
         25          32          14          16          21          89 
  ENMD-2076  Fedratinib Gedatolisib    I-BET151 Ipatasertib         JQ1 
         12          22          24          19          34          25 
  LY3009120   Milciclib   Niraparib  NVP-BGT226  NVP-LCL161      OTS167 
         17          24          33          25          28          21 
 Pacritinib   PD0325901 PF-00477736 PF-03758309  Pictilisib Pracinostat 
         20          27          25          21          22          23 
Quisinostat Ralimetinib   SCH772984   Selinexor     SNS-032 Talazoparib 
         13          28          24          18          23          26 
    TGX-221      UCN-01  Volasertib     ZSTK474 
         27          33          22          31

AR-42      AT 101      AT9283    AZD-8186     AZD8055  Belinostat 
         26           3          25          20          30          23 
    BI 2536  Birabresib  Birinapant  BMS-754807  Buparlisib Cobimetinib 
         20          32          35          23          16          25 
 Copanlisib  Dactolisib  Danusertib   Dasatinib  Dinaciclib        DMSO 
         25          26          22          30          22          85 
  ENMD-2076  Fedratinib Gedatolisib    I-BET151 Ipatasertib         JQ1 
          9          16          38          30          25          32 
  LY3009120   Milciclib   Niraparib  NVP-BGT226  NVP-LCL161      OTS167 
         25          17          21          20          32          22 
 Pacritinib   PD0325901 PF-00477736 PF-03758309  Pictilisib Pracinostat 
         17          28          21          18          22          24 
Quisinostat Ralimetinib   SCH772984   Selinexor     SNS-032 Talazoparib 
         28          29          26          16          21          30 
    TGX-221      UCN-01  Volasertib     ZSTK474 
         25          37          19          32

Now we can iterate in them. The first element is always DMSO, and we store it. Also, it is the biggest one.

In [4]:
JHOS2_pseudo_DMSO <- JHOS2_pseudo$DMSO
PDC2_pseudo_DMSO <- PDC2_pseudo$DMSO
PDC3_pseudo_DMSO <- PDC3_pseudo$DMSO

Function for differential expression analysis

In [5]:
edgeR_diffExpress_PBRs <- function(control_dataset, drug_dataset, drug){
    
    # Binding control and drug dataset
    dataset <- cbind(control_dataset, drug_dataset) 
    
    # Groups identification
    groups <- sapply(colnames(dataset), function(x) strsplit(x = x, split = "_")[[1]][2]) 
                     
    # Instead of the drug name, we will put "Treatment" to make the function generalizable
    groups[groups != "DMSO"] <- "Treatment" 
    groups[groups == "DMSO"] <- "Control"                 
    groups <- factor(groups, levels = c("Control", "Treatment")) # Needs to be fixed to DMSO and drug
    
    # Creating a DGE list object                 
    dge_obj <- DGEList(counts = dataset, group = groups, remove.zeros = TRUE) 
                     
    # Average log CPM to perform filterings based on the previous histograms                 
    avglogCPM <- aveLogCPM(dge_obj)          
    keep <- (avglogCPM > 1)
    dge_obj_filtered <- dge_obj[keep, ]
                     
    # Normalization                 
    dge_obj_filtered <- calcNormFactors(dge_obj_filtered)
                                 
    # Preparing the design matrix and estimating the NB dispersions               
    design <- model.matrix(~0 + groups)
    colnames(design) <- levels(groups)
    dge_obj_filtered <- estimateDisp(dge_obj_filtered, design = design, robust = TRUE)
    
    # Fitting the model
    fit <- glmQLFit(dge_obj_filtered, design = design)
                     
    # Running the test, specifying the comparison
    # Positive logFC = higher in treatment
    qlf <- glmQLFTest(fit, contrast = makeContrasts(DrugVsDMSO = Treatment - Control, levels = design)) 
                     
    # Return results, BH p-value adjustment                 
    tt <- topTags(qlf, n = Inf)
    return(tt$table)
}

Differential expression, 4 cores max as RAM goes high fast and we are running other analyses in parallel on the same cluster

In [6]:
drug_names <- names(JHOS2_pseudo)[-which(names(JHOS2_pseudo) == "DMSO")]
drug_names

[1] "AR-42"       "AT 101"      "AT9283"      "AZD-8186"    "AZD8055"    
 [6] "Belinostat"  "BI 2536"     "Birabresib"  "Birinapant"  "BMS-754807" 
[11] "Buparlisib"  "Cobimetinib" "Copanlisib"  "Dactolisib"  "Danusertib" 
[16] "Dasatinib"   "Dinaciclib"  "ENMD-2076"   "Fedratinib"  "Gedatolisib"
[21] "I-BET151"    "Ipatasertib" "JQ1"         "LY3009120"   "Milciclib"  
[26] "Niraparib"   "NVP-BGT226"  "NVP-LCL161"  "OTS167"      "Pacritinib" 
[31] "PD0325901"   "PF-00477736" "PF-03758309" "Pictilisib"  "Pracinostat"
[36] "Quisinostat" "Ralimetinib" "SCH772984"   "Selinexor"   "SNS-032"    
[41] "Talazoparib" "TGX-221"     "UCN-01"      "Volasertib"  "ZSTK474"

In [7]:
JHOS2_diffexpress <- mclapply(X = drug_names, 
                        function(x) edgeR_diffExpress_PBRs(JHOS2_pseudo_DMSO, JHOS2_pseudo[[x]], x), 
                        mc.cores = 4)
names(JHOS2_diffexpress) <- drug_names
saveRDS(object = JHOS2_diffexpress, file = "JHOS2_PBRs_diffexpress_subsampled.RDS")

In [8]:
PDC2_diffexpress <- mclapply(X = drug_names, function(x) edgeR_diffExpress_PBRs(PDC2_pseudo_DMSO, PDC2_pseudo[[x]], x), 
                        mc.cores = 4)
names(PDC2_diffexpress) <- drug_names
saveRDS(object = PDC2_diffexpress, file = "PDC2_PBRs_diffexpress_subsampled.RDS")

In [9]:
PDC3_diffexpress <- mclapply(X = drug_names, function(x) edgeR_diffExpress_PBRs(PDC3_pseudo_DMSO, PDC3_pseudo[[x]], x), 
                        mc.cores = 4)
names(PDC3_diffexpress) <- drug_names
saveRDS(object = PDC3_diffexpress, file = "PDC3_PBRs_diffexpress_subsampled.RDS")

In [10]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 8.8 (Green Obsidian)

Matrix products: default
BLAS/LAPACK: /homedir01/adini22/.conda/envs/cellhashing_analyses/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] edgeR_3.40.2 limma_3.54.2

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.10     uuid_1.1-0      lattice_0.20-45 rlang_1.1.0    
 [5] fastmap_1.1.1   fansi_1.0.4     tools_4.2.2     grid_4.2.2     
 [9] utf8_1.2.3      cli_3.6.0       htmltools_0